In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, listdir, stat, chdir, system
from os.path import join, basename, exists, dirname, isfile, isdir

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

from epiweeks import Week
import numpy as np
from random import randint
from math import floor, ceil
import codecs

from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
from bulletin import default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
import pyminizip as pz

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
meses = {1:'Janeiro',2:'Fevereiro',3:'Março',4:'Abril',5:'Maio',6:'Junho',7:'Julho',8:'Agosto',9:'Setembro',10:'Outubro',11:'Novembro',12:'Dezembro'}

## 1. Atualização de arquivos e Carregamento das Bases de Dados

In [ ]:
# DOWNLOAD RT_PCR
mb = Metabase()

rt_pcr_path = join('C:\\', 'SESA', 'covid19datascience-dev', 'output', 'dash', 'rt_pcr.csv')
with open(join(mb.sql_path,'RT_PCR.sql')) as file:
        raw_sql = trim_overspace(" ".join([ row.replace('\n',' ') for row in file.readlines() if not '--' in row ]))

mb.download(raw_sql, rt_pcr_path)



# DOWNLOAD TESTES RÁPIDOS DIÁRIO
mb = Metabase()

teste_rapido_path = join('C:\\', 'SESA', 'covid19datascience-dev', 'input', 'queries', 'testes_rapidos_diario.csv')
with open(join(mb.sql_path,'testes_rapidos.sql')) as file:
        raw_sql = trim_overspace(" ".join([ row.replace('\n',' ') for row in file.readlines() if not '--' in row ]))

mb.download(raw_sql, teste_rapido_path)

In [ ]:
# municipios = Municipios()
# municipios['mun_resid'] = municipios['municipio']
# municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

# municipio = municipios.copy()

In [ ]:
municipios = Municipios()

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)

In [ ]:
casos_confirmados = cc.df.copy()

# casos_confirmados = pd.merge(casos_confirmados.rename(columns={'ibge_residencia':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_residencia'})

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casos_confirmados = casos_confirmados.join(municipios.set_index('ibge'), on='ibge_residencia')

casos_confirmados['municipio_pr'] = casos_confirmados['municipio']
casos_confirmados.loc[casos_confirmados['uf_residencia']!='PR','municipio_pr'] = 'Fora'

casos_confirmados.loc[casos_confirmados['uf_residencia']!='PR','ibge7'] = 9999999

casos_confirmados['uf_pr'] = 'PR'
casos_confirmados.loc[casos_confirmados['uf_residencia']!='PR','uf_pr'] = 'Fora'

## 2. Quantitativos totais e dados enviados para o Ministério (COE, CONASS)

In [ ]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casos_confirmados.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,4,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'ÓBITOS',3:'ATIVOS',4:'ÓBITOS POR OUTRAS CAUSAS'})
total.index.name = None
total.columns.name = None

total.iloc[:4,2:].to_excel(join(default_output, 'relatorios', 'formulario_MS.xlsx'))

total

In [ ]:
total_macros = casos_confirmados.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'ÓBITOS',3:'ATIVOS',4:'ÓBITOS POR OUTRAS CAUSAS'})
total_macros = total_macros[['CASOS','ÓBITOS','RECUPERADOS','ATIVOS','ÓBITOS POR OUTRAS CAUSAS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

In [ ]:
total_regionais = casos_confirmados.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS',4:'OBITO POR OUTRAS CAUSAS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS','OBITO POR OUTRAS CAUSAS']]

total_regionais.astype(float)

In [ ]:
total_municipios = casos_confirmados.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'ÓBITOS',3:'ATIVOS',4:'ÓBITOS POR OUTRAS CAUSAS'})
total_municipios = total_municipios[['CASOS','ÓBITOS','RECUPERADOS','ATIVOS','ÓBITOS POR OUTRAS CAUSAS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

In [ ]:
ministerio_cols = ['municipio_pr','macro','CASOS','ÓBITOS']

ministerio_sort = pd.read_excel(join(default_input,'base',"Casos confirmados covid19 no PR por município.xlsx"), skiprows=1).fillna('').set_index(['Município de residência','macro'])

ministerio = total_municipios.reset_index()[ministerio_cols]
# ministerio = pd.merge(ministerio_sort[['ibge7']], total_municipios.reset_index()[ministerio_cols], how='left', on='ibge7' )
# del ministerio['ibge7']

ministerio['macro'] = ministerio['macro'].str.upper()

ministerio.loc[ministerio['municipio_pr']=='Fora',['municipio_pr']] = 'Fora do Parana'
ministerio.loc[ministerio['macro']=='FORA',['macro']] = ''

ministerio.loc[400,'municipio_pr'] = 'TOTAL GERAL'
ministerio.loc[400,'macro'] = ''
ministerio.loc[400,'CASOS'] = total.loc['CASOS','GERAL']
ministerio.loc[400,'ÓBITOS'] = total.loc['ÓBITOS','GERAL']

ministerio.columns = ['Município de residência', 'macro', 'Casos confirmados', 'Óbitos']

ministerio_sort.update(ministerio.set_index(['Município de residência','macro']))

ministerio = ministerio_sort.reset_index()

writer = pd.ExcelWriter(join(default_output,'relatorios',"Casos confirmados covid19 no PR por municipio.xlsx"),
                    engine='xlsxwriter')

ministerio.to_excel(writer,'total_municipios', index=False)
fit_cols(writer,ministerio,'total_municipios')

writer.save()

from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill

wb = load_workbook(filename = join(default_output,'relatorios',"Casos confirmados covid19 no PR por municipio.xlsx"))
sheet = wb['total_municipios']
sheet.insert_rows(1)
sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=4)
cell = sheet.cell(row=1, column=1)  
cell.value = 'Casos confirmados covid-19 no Paraná'  
cell.alignment = Alignment(horizontal='center', vertical='center')  
wb.save(filename = join(default_output,'relatorios',"Casos confirmados covid19 no PR por municipio.xlsx"))

ministerio

In [ ]:
# SOMA para verificação
ministerio.iloc[:-1,2:].sum()

In [ ]:
if (ministerio.iloc[:-1,2:].sum()[0] != total.iloc[:-3,2:].values[0]):
    raise Exception(f"Conferir Relatório que será enviado ao MS")

In [ ]:
#AJUSTA A PLANILHA DE TOTAIS MUNICÍPIOS PARA AS REGIONAIS
total_municipios = total_municipios.droplevel(axis=0, level='macro')
total_municipios = total_municipios.sort_index(axis=0, level='rs', ascending=True)

## 3. Dados Fontes

In [ ]:
path_verify = join(default_output, 'dash', 'Laboratorios.csv')
if (not exists(path_verify)):
    raise Exception(f"Os dados do GAL não foram atualizados. Para prosseguir com os Dados Fontes, execute o notebook GAL primeiro!")

In [ ]:
ddf1 = cc.dados_fontes()
ddf1.to_csv(join(default_output,'dash', 'Dados_Fonte.csv'), index = False, sep = ';')

In [ ]:
ddf2 = cc.dados_fontes2()
ddf2.to_csv(join(default_output,'dash', 'Dados_Fonte2.csv'), index = False, sep = ';', decimal=',', float_format='%.3f')
print(len(ddf2))

In [ ]:
ddf3 = cc.dados_fontes3()
ddf3.loc[ddf3['IBGE_RES_PR']=='NOROESTE','IBGE_RES_PR'] = 'NOROEST'
ddf3.loc[ddf3['Tipo_Data']=='Inicio_Sintomas','data'] = ddf3.loc[ddf3['Tipo_Data']=='Inicio_Sintomas','data'].apply(pd.to_datetime).dt.date
ddf3.to_csv(join(default_output,'dash', 'Dados_Fonte3.csv'), index = False, sep = ';')
print(len(ddf3))

### 3.1. Novos Casos e Óbitos por ano e mês

In [ ]:
casos_confirmadosPR = cc.df.loc[cc.df['uf_residencia'] == 'PR']

writer = pd.ExcelWriter(join(default_output,'dash', 'novos_casos_obitos_por_mes.xlsx'))

novos_casos_mes = casos_confirmadosPR.loc[casos_confirmadosPR['data_comunicacao']==today].groupby([(casos_confirmadosPR['data_diagnostico'].rename('ano').dt.year),(casos_confirmadosPR['data_diagnostico'].rename('mês').dt.month)])[['identificacao']].count().rename(columns={'identificacao':'qtde'}).reset_index().astype(int).replace({'mês':meses}).set_index(['ano','mês'])
novos_casos_mes.to_excel(writer,'novos_casos')
display(novos_casos_mes)

novos_obitos_mes = casos_confirmadosPR.loc[(casos_confirmadosPR['evolucao']== 2 ) & (casos_confirmadosPR['data_com_evolucao']==today)].groupby([(casos_confirmadosPR['data_cura_obito'].rename('ano').dt.year),(casos_confirmadosPR['data_cura_obito'].rename('mês').dt.month)])[['identificacao']].count().rename(columns={'identificacao':'qtde'}).reset_index().astype(int).replace({'mês':meses}).set_index(['ano','mês'])
if len(novos_obitos_mes) == 0:
    novos_obitos_mes.loc[('',''),'qtde'] = 0

novos_obitos_mes.to_excel(writer,'novos_obitos')
display(novos_obitos_mes)

writer.save()

## 4. Arquivos enviados para COMUNICAÇÃO

### 4.1. Definições de variáveis importantes antes de alterar o cc.df

In [ ]:
casosPR = cc.df.loc[(cc.df['uf_residencia']=='PR')].copy()
casosPR['ano_caso'] = casosPR['data_diagnostico'].dt.year
casosPR['mes_caso'] = casosPR['data_diagnostico'].dt.month


obitosPR = cc.df.loc[(cc.df['evolucao']==2) & (cc.df['uf_residencia']=='PR')].copy()
obitosPR = obitosPR.join(Municipios().set_index('ibge'), on='ibge_residencia')
obitosPR['ano_obito'] = obitosPR['data_cura_obito'].dt.year
obitosPR['mes_obito'] = obitosPR['data_cura_obito'].dt.month

### 4.2. Geração dos arquivos .csv enviados

In [ ]:
cc.df['origem_nome'] = cc.replace('origem',inplace=False)

municipioss = Municipios()
municipioss['rs'] = municipioss['rs'].astype('str').apply(lambda x: x.zfill(2))
municipioss['mun_resid'] = municipioss['municipio'].apply(normalize_text)     
municipioss.loc[municipioss['uf']!='PR','mun_resid'] += '/' + municipios['uf']

df_casos = cc.df.copy()
df_casos = pd.merge(df_casos.rename(columns={'ibge_residencia':'ibge'}),municipioss[['ibge','ibge7','rs','mun_resid','uf','municipio', 'macro']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia', 'ibge7':'ibge7_resid'})
df_casos = pd.merge(df_casos.rename(columns={'ibge_unidade_notifica':'ibge'}),municipioss[['ibge','ibge7','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica', 'ibge7':'ibge7_atend'})

In [ ]:


casos_obitos = cc.comunicacao_casosObitos(df_casos)
casos_obitos.to_csv(join(default_output, 'comunicacao',  f"INFORME_EPIDEMIOLOGICO_{hoje.strftime('%d_%m_%Y')}_OBITOS_CASOS_MUNICIPIO.csv"), index=False, sep=';',encoding='utf-8-sig')

In [ ]:


csv_geral_old = cc.comunicacao_csvGeralOld(df_casos)
csv_geral_old.to_csv(join(default_output, 'comunicacao',  f"INFORME_EPIDEMIOLOGICO_{hoje.strftime('%d_%m_%Y')}_GERAL.csv"), index=False, sep=';',encoding='utf-8-sig')

In [ ]:
# 

csv_geral_new = cc.comunicacao_csvGeralNew()
csv_geral_new.to_csv(join(default_output, 'comunicacao',  f"INFORME_EPIDEMIOLOGICO_{hoje.strftime('%d_%m_%Y')}_COMPLETO.csv"), index=False, sep=';',encoding='utf-8-sig')
csv_geral_new.to_csv(join(default_output, 'enviar_celepar',  f"CORONAVIRUS.INFORME_EPIDEMIOLOGICO_COMPLETO.csv"), index=False, sep=';',encoding='utf-8-sig')

In [ ]:
writer = pd.ExcelWriter(join(default_output,'comunicacao', f"MunicipiosSemObitos_CasosObitosMesDia_{hoje.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')


writer_dash = pd.ExcelWriter(join(default_output,'dash', f"casos_obitos_mes.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

In [ ]:
# MUNICÍPIOS SEM ÓBITOS
# obitosPR = obitosPR.sort_values('data_com_evolucao')
# obitos_dia = obitosPR[['rs','municipio','data_com_evolucao']].groupby(['rs','municipio']).last()
# obitos_dia['dias_apos_ultimo_obito'] = today - obitos_dia['data_com_evolucao']
# obitos_dia = obitos_dia.sort_values('dias_apos_ultimo_obito', ascending=False)
# obitos_dia.columns = ['data último óbito', 'dias após último óbito']
# obitos_dia['ano/mês último óbito'] = obitos_dia['data último óbito'].dt.strftime('%Y/%m')
# obitos_dia['data último óbito'] = obitos_dia['data último óbito'].dt.date
# obitos_dia.to_excel(writer, 'municipios_comunicacao')

In [ ]:
# MUNICÍPIOS SEM ÓBITOS
obitosPR = obitosPR.sort_values('data_cura_obito')
obitos_dia = obitosPR[['rs','municipio','data_cura_obito']].groupby(['rs','municipio']).last()
obitos_dia['dias_apos_ultimo_obito'] = today - obitos_dia['data_cura_obito']
obitos_dia = obitos_dia.sort_values('dias_apos_ultimo_obito', ascending=False)
obitos_dia.columns = ['data último óbito', 'dias após último óbito']
obitos_dia['ano/mês último óbito'] = obitos_dia['data último óbito'].dt.strftime('%Y/%m')
obitos_dia['data último óbito'] = obitos_dia['data último óbito'].dt.date
obitos_dia.to_excel(writer, 'municipios_sem_obitos')

In [ ]:
# CASOS E ÓBITOS POR MÊS
tabela_casos_mes = casosPR.groupby(['ano_caso', 'mes_caso'])[['id_notifica']].count().rename(columns={'id_notifica':'casos'})
tabela_casos_mes.rename(index=meses, inplace=True)

tabela_obitos_mes = obitosPR.groupby(['ano_obito', 'mes_obito'])[['id_notifica']].count().rename(columns={'id_notifica':'óbitos'})
tabela_obitos_mes.rename(index=meses, inplace=True)


tabela_casos_mes.index.set_names(['ano', 'mes'], inplace=True)
tabela_obitos_mes.index.set_names(['ano', 'mes'], inplace=True)
tabela_mes_PR = tabela_casos_mes.join(tabela_obitos_mes)

tabela_mes_PR['% do total de casos'] = tabela_mes_PR['casos'] / tabela_mes_PR['casos'].sum()
tabela_mes_PR['% do total de óbitos'] = tabela_mes_PR['óbitos'] / tabela_mes_PR['óbitos'].sum()

#salvar
tabela_mes_PR.to_excel(writer, 'casos_obitos_mes')
tabela_mes_PR.reset_index().to_excel(writer_dash, 'casos_obitos_mes')

In [ ]:
# CASOS E ÓBITOS POR DIA
tabela_casos_dia = casosPR.groupby(['ano_caso', 'mes_caso', 'data_diagnostico'])[['id_notifica']].count().rename(columns={'id_notifica':'casos'})
tabela_casos_dia.rename(index=meses, inplace=True)

tabela_obitos_dia = obitosPR.groupby(['ano_obito', 'mes_obito', 'data_cura_obito'])[['id_notifica']].count().rename(columns={'id_notifica':'óbitos'})
tabela_obitos_dia.rename(index=meses, inplace=True)

tabela_casos_dia.index.set_names(['ano', 'mes', 'data'], inplace=True)
tabela_obitos_dia.index.set_names(['ano', 'mes', 'data'], inplace=True)
tabela_dia_PR = tabela_casos_dia.join(tabela_obitos_dia)

for i in tabela_dia_PR.columns:
    tabela_dia_PR[i] = tabela_dia_PR[i].fillna(0)

#salvar
tabela_dia_PR.to_excel(writer, 'casos_obitos_dia')

In [ ]:
writer.save()
writer.close()
writer_dash.save()
writer_dash.close()

In [ ]:
# GERA ARQUIVOS COMPACTADOS DE HOJE
dirpath = join('C:\\', 'SESA', 'covid19datascience-dev','output', 'comunicacao')
archives = [ f for f in listdir(dirpath) if isfile(join(dirpath, f)) ]

chdir(dirpath)

pz.compress_multiple(archives, [], join(dirpath, f"comunicacao_{hoje.strftime('%d_%m_%Y')}.zip"), None, 9)

chdir(current_dir)


# # REMOVE ARQUIVOS DE HOJE
# for i in archives:
#     try:
#         remove(join(dirpath, i))
#     except:
#         pass

## 5. Arquivos enviados para as REGIONAIS

In [ ]:
# if ((hoje.day_name() == 'Saturday')):
#     raise

In [ ]:
used_cols = ['id_notifica', 'paciente', 'sexo', 'idade', 'nome_mae', 'uf_residencia', 'ibge_residencia', 'municipio', 'ibge_unidade_notifica', 'data_1o_sintomas', 'data_diagnostico', 'data_comunicacao', 'evolucao', 'data_cura_obito', 'data_com_evolucao', 'numero_do', 'macro', 'rs', 'regional', 'reinfeccao', 'reinfeccao_ids']
casos_confirmados = casos_confirmados.drop(columns=[col for col in casos_confirmados.columns if not col in used_cols])


# AJUSTE DE CASOS
casos_confirmados['regional'] = casos_confirmados['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ 'Regional'
casos_confirmados['macro'] = casos_confirmados['macro'].apply(lambda x: macros_idx_labels[x])

casos_confirmados['ano_caso'] = casos_confirmados['data_diagnostico'].dt.year
casos_confirmados['mes_caso'] = casos_confirmados['data_diagnostico'].dt.month
casos_confirmados['SE_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: Week.fromdate(x))


# casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
# casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'ÓBITO'
# casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'
# casos_confirmados.loc[casos_confirmados['evolucao']==4,'evolucao'] = 'ÓBITO POR OUTRAS CAUSAS'
dict_evolucao = {'evolucao': {1: 'CURA', 2: 'ÓBITO', 3: 'ATIVO', 4: 'ÓBITO POR OUTRAS CAUSAS'}}
casos_confirmados.replace(dict_evolucao, inplace=True)


casos_confirmados.loc[casos_confirmados['reinfeccao']==True,'reinfeccao'] = 'Provável caso de REINFECÇÃO'
casos_confirmados.loc[casos_confirmados['reinfeccao']==False,'reinfeccao'] = None

casos_confirmados.loc[(casos_confirmados['evolucao']=='ÓBITO') & (casos_confirmados['numero_do'].isna()), 'numero_do'] = 'DO NÃO INSERIDA NO NOTIFICA COVID-19'




# AJUSTE DE ÓBITOS
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='ÓBITO'].copy()
obitos_confirmados['ano_obito'] = obitos_confirmados['data_cura_obito'].dt.year
obitos_confirmados['mes_obito'] = obitos_confirmados['data_cura_obito'].dt.month
obitos_confirmados['SE_obito'] = obitos_confirmados['data_cura_obito'].apply(lambda x: Week.fromdate(x))

In [ ]:
# 
# writer = pd.ExcelWriter(join(default_output,'regionais', f"casos_gerais_{hoje.strftime('%d_%m_%Y')}.xlsx"),
#                     engine='xlsxwriter',
#                     datetime_format='dd/mm/yyyy',
#                     date_format='dd/mm/yyyy')

# total.to_excel(writer,'total')
# total_macros.to_excel(writer,'total_macros')
# total_regionais.to_excel(writer,'total_regionais')
# total_municipios.to_excel(writer,'total_municipios')

# fit_cols(writer,total,'total')
# fit_cols(writer,total_macros,'total_macros')
# fit_cols(writer,total_regionais,'total_regionais')
# fit_cols(writer,total_municipios,'total_municipios')

# for macro in tqdm(list(macros_idx_labels.values())):
# #     print(macros_idx_labels[macro])
#     casos_macro = casos_confirmados.loc[casos_confirmados['macro']==macro]#.copy()
# #     casos_macro['macro'] = casos_macro['macro'].apply(lambda x: macros_idx_labels[x])
#     casos_macro = casos_macro.sort_values(by='data_diagnostico', ascending=False)
#     casos_macro.to_excel(writer,macro,index=False)
#     fit_cols(writer,casos_macro,macro)


# writer.save()
# writer.close()

In [ ]:

writer = pd.ExcelWriter(join(default_output,'regionais', f"obitos_gerais_{hoje.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

# obitos_confirmados['macro'] = obitos_confirmados['macro'].apply(lambda x: macros_idx_labels[x])

obitos_confirmados = obitos_confirmados.sort_values(by='data_cura_obito', ascending=False)
obitos_confirmados.to_excel(writer,'obitos',index=False)
fit_cols(writer,obitos_confirmados,'obitos')


writer.save()
writer.close()

In [ ]:

writer = pd.ExcelWriter(join(default_output,'regionais',f"casos_regionais_{hoje.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional]
#     casos_regional['macro'] = casos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    casos_regional = casos_regional.sort_values(by='data_diagnostico', ascending=False)
    casos_regional.to_excel(writer,regional, index=False)
    fit_cols(writer,casos_regional,regional)

writer.save()
writer.close()

In [ ]:

writer = pd.ExcelWriter(join(default_output,'regionais',f"obitos_regionais_{hoje.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional]
#     obitos_regional['macro'] = obitos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    obitos_regional = obitos_regional.sort_values(by='data_cura_obito', ascending=False)
    obitos_regional.to_excel(writer,regional, index=False)
    fit_cols(writer,obitos_regional,regional)

writer.save()
writer.close()

In [ ]:
# GERA ARQUIVOS COMPACTADOS DE HOJE
paths = []


# paths = [join(default_output,'regionais', f"casos_gerais_{hoje.strftime('%d_%m_%Y')}.xlsx")]
paths.append(join(default_output,'regionais', f"obitos_gerais_{hoje.strftime('%d_%m_%Y')}.xlsx"))
paths.append(join(default_output,'regionais', f"casos_regionais_{hoje.strftime('%d_%m_%Y')}.xlsx"))
paths.append(join(default_output,'regionais', f"obitos_regionais_{hoje.strftime('%d_%m_%Y')}.xlsx"))

for path in paths:
    pz.compress(path, None, join(default_output,'regionais', f"{basename(path)}.zip".replace(".xlsx", "")), "Jt\j+ZXVOAbSw*i&J)ZePVGiQgU4wc", 9)


# REMOVE ARQUIVOS XLSX DE HOJE
# remove(join(default_output,'regionais', f"casos_gerais_{hoje.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output,'regionais', f"obitos_gerais_{hoje.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output,'regionais', f"casos_regionais_{hoje.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output,'regionais', f"obitos_regionais_{hoje.strftime('%d_%m_%Y')}.xlsx"))

In [ ]:
casosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]
obitosPR = obitos_confirmados.loc[casos_confirmados['rs']!=99]

In [ ]:
used_cols = ['id_notifica', 'paciente', 'sexo', 'idade', 'nome_mae', 'uf_residencia', 'ibge_residencia', 'municipio', 'ibge_unidade_notifica', 'data_1o_sintomas', 'data_diagnostico', 'data_comunicacao', 'evolucao', 'data_cura_obito', 'data_com_evolucao', 'numero_do', 'macro', 'rs', 'regional', 'reinfeccao', 'reinfeccao_ids']


for regional in tqdm(casosPR['rs'].unique()):
    casosRS = casosPR.loc[casosPR['rs']==regional]
    obitosRS = obitosPR.loc[obitosPR['rs']==regional]
    
    
    writer = pd.ExcelWriter(join(default_output,'regionais','temp',f"{str(regional).zfill(2)}RS_{hoje.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')
    
    
    
    
    
    #----TOTAIS
    total.to_excel(writer,'total')
    total_macros.to_excel(writer,'total_macros')
    total_regionais.to_excel(writer,'total_regionais')
    total_municipios.loc[regional].to_excel(writer,'total_municipios')

    fit_cols(writer,total,'total')
    fit_cols(writer,total_macros,'total_macros')
    fit_cols(writer,total_regionais,'total_regionais')
    fit_cols(writer,total_municipios,'total_municipios')
    
    
    
    #----CASOS E ÓBITOS POR MES
    tabela_casos_mes = casosRS.groupby(['ano_caso', 'mes_caso'])[['id_notifica']].count().rename(columns={'id_notifica':'casos'})
    tabela_casos_mes.rename(index=meses, inplace=True)

    tabela_obitos_mes = obitosRS.groupby(['ano_obito', 'mes_obito'])[['id_notifica']].count().rename(columns={'id_notifica':'óbitos'})
    tabela_obitos_mes.rename(index=meses, inplace=True)


    tabela_casos_mes.index.set_names(['ano', 'mes'], inplace=True)
    tabela_obitos_mes.index.set_names(['ano', 'mes'], inplace=True)
    tabela_mes = tabela_casos_mes.join(tabela_obitos_mes)
    
    for i in tabela_mes.columns:
        tabela_mes[i] = tabela_mes[i].fillna(0)

    #salvar
    tabela_mes.to_excel(writer, 'casos_obitos_mes')
    
    
    
    #----CASOS E ÓBITOS POR SE
    tabela_casos_SE = casosRS.groupby(['SE_caso'])[['id_notifica']].count()
    tabela_casos_SE.reset_index(inplace=True)
    tabela_casos_SE.rename(columns={'SE_caso':'SE', 'id_notifica':'casos'}, inplace=True)


    tabela_obitos_SE = obitosRS.groupby(['SE_obito'])[['id_notifica']].count()
    tabela_obitos_SE.reset_index(inplace=True)
    tabela_obitos_SE.rename(columns={'SE_obito':'SE', 'id_notifica':'óbitos'}, inplace=True)


    tabela_SE = pd.merge(tabela_casos_SE, tabela_obitos_SE, on='SE', how='inner')

    tabela_SE['mes'] = tabela_SE['SE'].apply(lambda x: int(Week.startdate(x).strftime("%m")))
    tabela_SE['mes'] = tabela_SE['mes'].apply(lambda x: meses[x])
    tabela_SE['ano'] = tabela_SE['SE'].apply(lambda x: int(Week.startdate(x).strftime("%Y")))

    tabela_SE['SE'] = [ x.week for x in tabela_SE['SE'].values]

    tabela_SE.set_index(['ano','mes','SE'], inplace=True)
    
    for i in tabela_SE.columns:
        tabela_SE[i] = tabela_SE[i].fillna(0)
    
    #salvar
    tabela_SE.to_excel(writer, 'casos_obitos_SE')
    
    
    
    #----CASOS E ÓBITOS POR DIA
    tabela_casos_dia = casosRS.groupby(['ano_caso', 'mes_caso', 'data_diagnostico'])[['id_notifica']].count().rename(columns={'id_notifica':'casos'})
    tabela_casos_dia.rename(index=meses, inplace=True)

    tabela_obitos_dia = obitosRS.groupby(['ano_obito', 'mes_obito', 'data_cura_obito'])[['id_notifica']].count().rename(columns={'id_notifica':'óbitos'})
    tabela_obitos_dia.rename(index=meses, inplace=True)

    tabela_casos_dia.index.set_names(['ano', 'mes', 'data'], inplace=True)
    tabela_obitos_dia.index.set_names(['ano', 'mes', 'data'], inplace=True)
    tabela_dia = tabela_casos_dia.join(tabela_obitos_dia)
    
    #medias móveis
    tabela_dia['media_móvel_casos_7dias'] = tabela_dia['casos'].rolling(7).mean()
    # tabela_dia['variacao_media_movel_casos_14dias(%)'] = taias'].rolling(14).agg(lambda df: (df.iloc[-1] - df.iloc[0])/df.iloc[0] * 100)

    tabela_dia['media_móvel_óbitos_7dias'] = tabela_dia['óbitos'].rolling(7).mean()
    # tabela_dia['variacao_media_movel_obitos_14dias(%)'] = tabela_dia['media_móvel_casos_7dias'].rolling(14).agg(lambda df: (df.iloc[-1] - df.iloc[0])/df.iloc[0] * 100)
    tabela_dia = tabela_dia[['casos', 'media_móvel_casos_7dias', 'óbitos', 'media_móvel_óbitos_7dias']]
    
    arrays = [['casos', 'casos', 'óbitos', 'óbitos'], ['qte_casos', 'media_móvel_casos_7dias', 'qte_obitos', 'media_móvel_obitos_7dias']]
    tabela_dia = tabela_dia.rename(columns={'casos':'qte_casos', 'óbitos':'qte_óbitos'})
    tabela_dia.columns = pd.MultiIndex.from_arrays(arrays)
    
    
    for i in tabela_dia.columns:
        tabela_dia[i] = tabela_dia[i].fillna(0)

    #salvar
    tabela_dia.to_excel(writer, 'casos_obitos_dia')
    
    
    
    #----PLANILHA DE CASOS CONFIRMADOS DA REGIONAL
    casosRS = casosRS.drop(columns=[col for col in casosRS.columns if not col in used_cols])
    casosRS = casosRS.sort_values(by='data_diagnostico', ascending=False)
    casosRS.to_excel(writer,'casos_confirmados', index=False)
    fit_cols(writer,casosRS,'casos_confirmados')
    
    
    
    #----PLANILHA DE ÓBITOS CONFIRMADOS DA REGIONAL
    obitosRS = obitosRS.drop(columns=[col for col in obitosRS.columns if not col in used_cols])
    obitosRS = obitosRS.sort_values(by='data_cura_obito', ascending=False)
    obitosRS.to_excel(writer,'obitos_confirmados', index=False)
    fit_cols(writer,obitosRS,'obitos_confirmados')
    
    writer.save()
writer.close()

In [ ]:
# GERA ARQUIVOS COMPACTADOS DE HOJE
regionais_path = join(default_output,'regionais','temp')
archives = [ f for f in listdir(regionais_path) if isfile(join(regionais_path, f)) ]

chdir(regionais_path)


pz.compress_multiple(archives, [], join(default_output,'regionais', f"casos_obitos_regionais_{hoje.strftime('%d_%m_%Y')}.zip"), "Jt\j+ZXVOAbSw*i&J)ZePVGiQgU4wc", 9)

for files in archives:
    try:
        print('Removing file: ', files)
        remove(join(regionais_path, files))
    except:
        pass

## 6. Arquivo enviado para a DGS

In [ ]:
writer = pd.ExcelWriter(join(default_output,'dgs', f"casos_obitos_{hoje.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

In [ ]:
# CASOS POR SE E MACRO (PARANÁ)
macro_SE = casosPR.groupby(['SE_caso','macro'])[['id_notifica']].count()
macro_SE = macro_SE.unstack('macro').droplevel(0,1).reset_index()
macro_SE.rename(columns={'SE_caso':'SE', 'id_notifica':'casos'}, inplace=True)


macro_SE['mes'] = macro_SE['SE'].apply(lambda x: int(Week.startdate(x).strftime("%m")))
macro_SE['mes'] = macro_SE['mes'].apply(lambda x: meses[x])
macro_SE['ano'] = macro_SE['SE'].apply(lambda x: int(Week.startdate(x).strftime("%Y")))

macro_SE['SE'] = [ x.week for x in macro_SE['SE'].values]

macro_SE.set_index(['ano','mes','SE'], inplace=True)

macro_SE.columns.name = None

for i in macro_SE.columns:
    macro_SE[i] = macro_SE[i].fillna(0).astype(int)

macro_SE['PR'] = macro_SE.sum(axis=1)
macro_SE = macro_SE[['PR', 'Leste', 'Oeste', 'Noroeste', 'Norte']]
macro_SE_casos = macro_SE.copy()

macro_SE_casos.to_excel(writer,'casos')
macro_SE_casos

In [ ]:
# ÓBITOS POR SE E MACRO
macro_SE = obitosPR.groupby(['SE_obito','macro'])[['id_notifica']].count()
macro_SE = macro_SE.unstack('macro').droplevel(0,1).reset_index()
macro_SE.rename(columns={'SE_obito':'SE', 'id_notifica':'obitos'}, inplace=True)


macro_SE['mes'] = macro_SE['SE'].apply(lambda x: int(Week.startdate(x).strftime("%m")))
macro_SE['mes'] = macro_SE['mes'].apply(lambda x: meses[x])
macro_SE['ano'] = macro_SE['SE'].apply(lambda x: int(Week.startdate(x).strftime("%Y")))

macro_SE['SE'] = [ x.week for x in macro_SE['SE'].values]

macro_SE.set_index(['ano','mes','SE'], inplace=True)

macro_SE.columns.name = None

for i in macro_SE.columns:
    macro_SE[i] = macro_SE[i].fillna(0).astype(int)

macro_SE['PR'] = macro_SE.sum(axis=1)
macro_SE = macro_SE[['PR', 'Leste', 'Oeste', 'Noroeste', 'Norte']]
macro_SE_obitos = macro_SE.copy()

macro_SE_obitos.to_excel(writer,'obitos')
macro_SE_obitos

In [ ]:
# CASOS E ÓBITOS POR MÊS
tabela_mes_PR.to_excel(writer,'mes')

writer.save()
writer.close()

## 7. ENVIA OS ARQUIVOS PARA O DRIVE

In [ ]:
# path = join('C:\\', 'SESA', 'covid19datascience-dev', 'scripts_python', '7-upload_diario.py')
# exec(open(path).read())

## 8. OUTROS ARQUIVOS

In [ ]:
# CASOS E ÓBITOS POR SE (PARANÁ)
tabela_casos_SE = casosPR.groupby(['SE_caso'])[['id_notifica']].count()
tabela_casos_SE.reset_index(inplace=True)
tabela_casos_SE.rename(columns={'SE_caso':'SE', 'id_notifica':'casos'}, inplace=True)


tabela_obitos_SE = obitosPR.groupby(['SE_obito'])[['id_notifica']].count()
tabela_obitos_SE.reset_index(inplace=True)
tabela_obitos_SE.rename(columns={'SE_obito':'SE', 'id_notifica':'obitos'}, inplace=True)


tabela = pd.merge(tabela_casos_SE, tabela_obitos_SE, on='SE', how='inner')

tabela['mes'] = tabela['SE'].apply(lambda x: int(Week.startdate(x).strftime("%m")))
tabela['mes'] = tabela['mes'].apply(lambda x: meses[x])
tabela['ano'] = tabela['SE'].apply(lambda x: int(Week.startdate(x).strftime("%Y")))

tabela['SE'] = [ x.week for x in tabela['SE'].values]

tabela.set_index(['ano','mes','SE'], inplace=True)
tabela

In [ ]:
pd.options.display.float_format = None
casos_por_diag = cc.df.loc[cc.df['uf_residencia']=='PR',['id_notifica','data_diagnostico']].groupby('data_diagnostico').count().rename(columns={'id_notifica':'casos'})
casos_por_diag['media_movel'] = casos_por_diag['casos'].rolling(7).mean()
casos_por_diag['variacao_movel_da_media_movel'] = casos_por_diag['media_movel'].rolling(14).agg(lambda df: (df.iloc[-1] - df.iloc[0])/df.iloc[0])
# obitos_por_diag = cc.df.loc[(cc.df['uf_residencia']=='PR')&(cc.df['evolucao']==2),['id_notifica','data_cura_obito']].groupby('data_cura_obito').count().rename(columns={'id_notifica':'obitos'})
# obitos_por_diag['media_movel'] = obitos_por_diag['obitos'].rolling(7).mean()
# obitos_por_diag['variacao_movel_da_media_movel'] = obitos_por_diag['media_movel'].rolling(14).agg(lambda df: (df.iloc[-1] - df.iloc[0])/df.iloc[0])
# recuperados_por_diag = cc.df.loc[(cc.df['uf_residencia']=='PR')&(cc.df['evolucao']==1),['id_notifica','data_cura_obito']].groupby('data_cura_obito').count().rename(columns={'id_notifica':'recuperados'})
# recuperados_por_diag['media_movel'] = recuperados_por_diag['recuperados'].rolling(7).mean()
# recuperados_por_diag['variacao_movel_da_media_movel'] = recuperados_por_diag['media_movel'].rolling(14).agg(lambda df: (df.iloc[-1] - df.iloc[0])/df.iloc[0])
# ativos_por_diag = cc.df.loc[(cc.df['uf_residencia']=='PR')&(cc.df['evolucao']==3),['id_notifica','data_cura_obito']].groupby('data_cura_obito').count().rename(columns={'id_notifica':'ativos'})
# ativos_por_diag['media_movel'] = ativos_por_diag['ativos'].rolling(7).mean()
# ativos_por_diag['variacao_movel_da_media_movel'] = ativos_por_diag['media_movel'].rolling(14).agg(lambda df: (df.iloc[-1] - df.iloc[0])/df.iloc[0])

# a = pd.merge(casos_por_diag,obitos_por_diag,how='outer',left_index=True,right_index=True,suffixes=['','_obitos'])
# a = pd.merge(a,recuperados_por_diag,how='outer',left_index=True,right_index=True,suffixes=['','_recuperados'])
# a = pd.merge(a,ativos_por_diag,how='outer',left_index=True,right_index=True,suffixes=['','_ativos'])
# a = a.fillna(0)
# a

In [ ]:
casos_por_diag

In [ ]:
ax = casos_por_diag[['casos','media_movel']].plot()
casos_por_diag[['variacao_movel_da_media_movel']].plot(secondary_y=True, ax=ax, ls=':')

In [ ]:
casos_por_diag['variacao_movel_da_media_movel'].apply(lambda x: x*100).plot()